# a. Import Package

In [1]:
import sys
import os
from keras.preprocessing.image import ImageDataGenerator
from keras import optimizers
from keras.models import Sequential
from keras.layers import Dropout, Flatten, Dense, Activation
from keras.layers.convolutional import Convolution2D, MaxPooling2D
from keras import callbacks

Using TensorFlow backend.


In [2]:
import torch

import torch.nn as nn
from torch.nn import init
import torchvision
from torchvision import transforms
import torch.optim as optim
from torch.utils.data import DataLoader
import torchvision.datasets as dset
from torchvision.datasets import ImageFolder

import numpy as np

In [60]:
imgsz = 32

normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],\
std=[0.229, 0.224, 0.225])

transform_test = transforms.Compose([transforms.Resize((imgsz, imgsz)), \
                      transforms.ToTensor(), \
                        normalize])

test_raw = ImageFolder('test', transform_test)

test_loader = DataLoader(test_raw, num_workers=2,
                          shuffle=False)

In [50]:
print(len(train_raw))

3333


# b. Create helper funtion

In [15]:
import torch.nn.functional as F  # useful stateless functions

#reshapes image data for use in a fully-connected neural network
def flatten(x):
    N = x.shape[0] # read in N, C, H, W
    return x.view(N, -1)

In [38]:
class Flatten(nn.Module):
    def forward(self, x):
        return flatten(x)
    
def init_weights(m):
    # print(m)
    if type(m) == nn.Conv2d or type(m) == nn.Linear:
        random_weight(m.weight.size())
        zero_weight(m.bias.size())

In [39]:
def check_accuracy_module(loader, model):
    num_correct = 0
    num_samples = 0
    model.eval()  # set model to evaluation mode
    
    with torch.no_grad():
        for x, y in loader:
            #x = x.to(device=device, dtype=dtype)  # move to device, e.g. GPU
            #y = y.to(device=device, dtype=torch.long)
            scores = model(x)
            _, preds = scores.max(1)
            num_correct += (preds == y).sum()
            num_samples += preds.size(0)
        acc = float(num_correct) / num_samples
        print('Got %d / %d correct (%.2f)' % (num_correct, num_samples, 100 * acc))

In [40]:
USE_GPU = True

dtype = torch.float32 # we will be using float throughout this tutorial

if USE_GPU and torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')

# Constant to control how frequently we print train loss
print_every = 10

print('using device:', device)

using device: cuda


# c. Build model structure

In [41]:
channel_input = 3
channel_1 = 1024
channel_2 = 512
channel_3 = 256
num_class = 70
learning_rate = 1e-3

model = nn.Sequential(
    
    # conv1
    nn.Conv2d(channel_input, channel_1, kernel_size=5, padding=2),
    
    nn.ReLU(),
    
    # conv2
    nn.Conv2d(channel_1, channel_2, kernel_size=3, padding=2),
    
    nn.ReLU(),
    
    # conv3
    nn.Conv2d(channel_2, channel_3, kernel_size=3, padding=1),
    
    nn.ReLU(),
    
    nn.MaxPool2d(2),
    
    Flatten(),
    
    # fc
    nn.Linear(channel_3*17*17, num_class)
)

In [42]:
model.load_state_dict(torch.load('new_trained_model.pt'))
model.eval()

Sequential(
  (0): Conv2d(3, 1024, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
  (1): ReLU()
  (2): Conv2d(1024, 512, kernel_size=(3, 3), stride=(1, 1), padding=(2, 2))
  (3): ReLU()
  (4): Conv2d(512, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (5): ReLU()
  (6): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (7): Flatten()
  (8): Linear(in_features=73984, out_features=70, bias=True)
)

# d. Show predicted class

In [55]:
def show_predict_label(loader, model): 
    pred_label = []
    
    with torch.no_grad():
        for x, y in loader:
            '''x = x.to(device=device, dtype=dtype)  # move to device, e.g. GPU
            y = y.to(device=device, dtype=torch.long)'''
            scores = model(x)
            _, preds = scores.max(1)
            pred_label.append(preds)
    predicted_class = np.array(pred_label)
    return predicted_class

In [61]:
predicted_class = show_predict_label(test_loader, model)
predicted_class_list = predicted_class.tolist()

tensor([38])
tensor([15])
tensor([36])
tensor([4])
tensor([14])
tensor([41])
tensor([9])
tensor([41])
tensor([4])
tensor([4])
tensor([2])
tensor([37])
tensor([1])
tensor([0])
tensor([21])
tensor([18])
tensor([15])
tensor([2])
tensor([1])
tensor([15])
tensor([68])
tensor([15])
tensor([15])
tensor([60])
tensor([15])
tensor([4])
tensor([40])
tensor([16])
tensor([32])
tensor([66])
tensor([15])
tensor([68])
tensor([17])
tensor([16])
tensor([14])
tensor([4])
tensor([27])
tensor([28])
tensor([18])
tensor([23])
tensor([40])
tensor([54])
tensor([8])
tensor([64])
tensor([14])
tensor([67])
tensor([15])
tensor([59])
tensor([15])
tensor([15])
tensor([2])
tensor([0])
tensor([17])
tensor([37])
tensor([49])
tensor([15])
tensor([18])
tensor([2])
tensor([15])
tensor([23])
tensor([60])
tensor([41])
tensor([0])
tensor([41])
tensor([30])
tensor([37])
tensor([24])
tensor([18])
tensor([15])
tensor([15])
tensor([15])
tensor([15])
tensor([42])
tensor([2])
tensor([37])
tensor([15])
tensor([2])
tensor([15])
tens

tensor([15])
tensor([2])
tensor([25])
tensor([37])
tensor([49])
tensor([30])
tensor([54])
tensor([60])
tensor([9])
tensor([8])
tensor([15])
tensor([32])
tensor([9])
tensor([8])
tensor([11])
tensor([24])
tensor([18])
tensor([16])
tensor([21])
tensor([51])
tensor([20])
tensor([9])
tensor([46])
tensor([16])
tensor([2])
tensor([21])
tensor([69])
tensor([20])
tensor([29])
tensor([7])
tensor([46])
tensor([38])
tensor([21])
tensor([5])
tensor([15])
tensor([16])
tensor([19])
tensor([2])
tensor([15])
tensor([16])
tensor([15])
tensor([15])
tensor([8])
tensor([35])
tensor([15])
tensor([2])
tensor([30])
tensor([18])
tensor([9])
tensor([15])
tensor([18])
tensor([4])
tensor([45])
tensor([16])
tensor([2])
tensor([17])
tensor([15])
tensor([46])
tensor([15])
tensor([21])
tensor([15])
tensor([31])
tensor([10])
tensor([7])
tensor([15])
tensor([32])
tensor([22])
tensor([41])
tensor([18])
tensor([56])
tensor([45])
tensor([53])
tensor([15])
tensor([15])
tensor([15])
tensor([60])
tensor([15])
tensor([0])
ten

tensor([68])
tensor([15])
tensor([64])
tensor([4])
tensor([15])
tensor([11])
tensor([15])
tensor([61])
tensor([15])
tensor([4])
tensor([58])
tensor([38])
tensor([19])
tensor([15])
tensor([18])
tensor([15])
tensor([68])
tensor([15])
tensor([2])
tensor([38])
tensor([49])
tensor([15])
tensor([2])
tensor([47])
tensor([58])
tensor([44])
tensor([69])
tensor([42])
tensor([29])
tensor([15])
tensor([15])
tensor([2])
tensor([15])
tensor([21])
tensor([21])
tensor([42])
tensor([2])
tensor([49])
tensor([14])
tensor([2])
tensor([15])
tensor([15])
tensor([15])
tensor([37])
tensor([37])
tensor([37])
tensor([15])
tensor([28])
tensor([26])
tensor([23])
tensor([10])
tensor([52])
tensor([48])
tensor([4])
tensor([37])
tensor([69])
tensor([38])
tensor([57])
tensor([57])
tensor([35])
tensor([42])
tensor([2])
tensor([23])
tensor([58])
tensor([18])
tensor([34])
tensor([2])
tensor([5])
tensor([15])
tensor([51])
tensor([64])
tensor([22])
tensor([66])
tensor([58])
tensor([41])
tensor([33])
tensor([22])
tensor([46

tensor([8])
tensor([62])
tensor([22])
tensor([15])
tensor([4])
tensor([23])
tensor([10])
tensor([2])
tensor([7])
tensor([15])
tensor([2])
tensor([15])
tensor([27])
tensor([52])
tensor([15])
tensor([26])
tensor([15])
tensor([17])
tensor([15])
tensor([66])
tensor([46])
tensor([23])
tensor([40])
tensor([17])
tensor([61])
tensor([2])
tensor([21])
tensor([3])
tensor([17])
tensor([2])
tensor([44])
tensor([34])
tensor([34])
tensor([20])
tensor([44])
tensor([45])
tensor([44])
tensor([15])
tensor([44])
tensor([31])
tensor([4])
tensor([16])
tensor([61])
tensor([66])
tensor([2])
tensor([51])
tensor([15])
tensor([37])
tensor([44])
tensor([17])
tensor([45])
tensor([66])
tensor([53])
tensor([42])
tensor([24])
tensor([22])
tensor([15])
tensor([19])
tensor([58])
tensor([15])
tensor([6])
tensor([68])
tensor([36])
tensor([46])
tensor([49])
tensor([10])
tensor([58])
tensor([19])
tensor([49])
tensor([2])
tensor([7])
tensor([18])
tensor([52])
tensor([37])
tensor([46])
tensor([44])
tensor([15])
tensor([4])


tensor([41])
tensor([43])
tensor([5])
tensor([2])
tensor([61])
tensor([9])
tensor([22])
tensor([65])
tensor([46])
tensor([39])
tensor([68])
tensor([11])
tensor([32])
tensor([29])
tensor([53])
tensor([15])
tensor([9])
tensor([63])
tensor([66])
tensor([19])
tensor([21])
tensor([15])
tensor([53])
tensor([15])
tensor([44])
tensor([45])
tensor([41])
tensor([15])
tensor([40])
tensor([4])
tensor([53])
tensor([15])
tensor([15])
tensor([42])
tensor([29])
tensor([26])
tensor([38])
tensor([42])
tensor([4])
tensor([2])
tensor([44])
tensor([44])
tensor([9])
tensor([11])
tensor([36])
tensor([25])
tensor([26])
tensor([4])
tensor([15])
tensor([26])
tensor([15])
tensor([26])
tensor([60])
tensor([34])
tensor([62])
tensor([15])
tensor([52])
tensor([38])
tensor([33])
tensor([61])
tensor([18])
tensor([19])
tensor([30])
tensor([40])
tensor([35])
tensor([32])
tensor([53])
tensor([23])
tensor([31])
tensor([9])
tensor([16])
tensor([46])
tensor([15])
tensor([37])
tensor([10])
tensor([9])
tensor([29])
tensor([2]

tensor([15])
tensor([60])
tensor([5])
tensor([26])
tensor([45])
tensor([26])
tensor([62])
tensor([40])
tensor([19])
tensor([18])
tensor([7])
tensor([2])
tensor([4])
tensor([59])
tensor([48])
tensor([40])
tensor([41])
tensor([19])
tensor([33])
tensor([2])
tensor([7])
tensor([13])
tensor([18])
tensor([4])
tensor([29])
tensor([18])
tensor([48])
tensor([15])
tensor([25])
tensor([26])
tensor([15])
tensor([15])
tensor([15])
tensor([33])
tensor([2])
tensor([40])
tensor([5])
tensor([43])
tensor([58])
tensor([36])
tensor([2])
tensor([17])
tensor([21])
tensor([4])
tensor([37])
tensor([62])
tensor([36])
tensor([18])
tensor([19])
tensor([0])
tensor([2])
tensor([15])
tensor([7])
tensor([48])
tensor([37])
tensor([2])
tensor([15])
tensor([15])
tensor([24])
tensor([18])
tensor([29])
tensor([15])
tensor([7])
tensor([52])
tensor([46])
tensor([15])
tensor([19])
tensor([3])
tensor([19])
tensor([4])
tensor([47])
tensor([15])
tensor([15])
tensor([53])
tensor([15])
tensor([15])
tensor([46])
tensor([61])
tens

In [62]:
print(predicted_class_list[:10])

[38, 15, 36, 4, 14, 41, 9, 41, 4, 4]


In [65]:
from sklearn.metrics import classification_report, accuracy_score, f1_score

# class target for each file in test/images directory
test_target = np.genfromtxt('plane_test_target.csv', delimiter=',').astype(int)

# classname (label) for 20 class plane dataset
id_target = np.load('ids_test.npy').item()

accuracy = accuracy_score(test_target, predicted_class_list)
print("Data Test Accuracy = %.3f%%" % (accuracy*100))

f1_macro = f1_score(test_target, predicted_class_list, average='macro')
f1_micro = f1_score(test_target, predicted_class_list, average='micro')
print("F1 Score: Macro = %.3f, Micro = %.3f" % (f1_macro,f1_micro))

Data Test Accuracy = 41.944%
F1 Score: Macro = 0.408, Micro = 0.419


In [66]:
# target_names = list(map(str, id_target.values()))
target_names = list(map(str, id_target.keys()))

print(classification_report(test_target, predicted_class_list, target_names=target_names))

                     precision    recall  f1-score   support

               A300       0.11      0.09      0.10        33
               A310       0.33      0.15      0.21        33
               A320       0.31      0.48      0.38       134
               A330       0.25      0.14      0.18        66
               A340       0.50      0.48      0.49       134
               A380       0.45      0.39      0.42        33
             ATR-42       0.35      0.36      0.36        33
             ATR-72       0.67      0.47      0.55        34
              An-12       0.58      0.64      0.61        33
            BAE 146       0.52      0.58      0.55        67
            BAE-125       0.75      0.55      0.63        33
    Beechcraft 1900       0.60      0.36      0.45        33
         Boeing 707       0.19      0.09      0.12        33
         Boeing 717       0.30      0.18      0.22        34
         Boeing 727       0.21      0.12      0.15        33
         Boeing 737    